<a href="https://colab.research.google.com/github/Veekypilia/Dataset-of-loraed-llama3/blob/main/Lora-llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1安装微调库 （2024/6/16修改-官方库变动，原视频中这段代码修改，现在只需要安装最新官方库和依赖）
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

#----------------------------------------------------------------------------------
#import torch
#major_version, minor_version = torch.cuda.get_device_capability()
# 由于Colab有torch 2.2.1，会破坏软件包，要单独安装
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#if major_version >= 8:
    # 新GPU，如Ampere、Hopper GPU（RTX 30xx、RTX 40xx、A100、H100、L40）。
#    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
#else:
    # 较旧的GPU（V100、Tesla T4、RTX 20xx）
#    !pip install --no-deps peft accelerate bitsandbytes
#    !pip install xformers==0.0.25  #最新的0.0.26不兼容
#pass

安装各种微调库，采用unsloth方法微调llama3-8b版本，在2024.06.17做了更新，发现了一些问题。

In [ ]:
#2加载模型
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unsloth: unsloth/llama-3-8b-bnb-4bit can only handle sequence lengths of at most 8192.
But with kaiokendev's RoPE scaling of 1.25, it can be magically be extended to 10240!


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#3微调前测试
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}
### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "请用中文回答", # instruction
        "现在有一个中文电子病历，请你帮我完成实体自动标注的任务，请注意，所需标注的实体类型为｛\"疾病和诊断\": 1, \"手术\": 2, \"解剖部位\": 3, \"药物\": 4, \"影像检查\": 5, \"实验室检查\": 6｝，电子病历的原文本为，患者3月前因“直肠癌”于在我院于全麻上行直肠癌根治术（DIXON术），手术过程顺利，术后给予抗感染及营养支持治疗，患者恢复好，切口愈合良好。，术后病理示：直肠腺癌（中低度分化），浸润溃疡型，面积3.5*2CM，侵达外膜。双端切线另送“近端”、“远端”及环周底部切除面未查见癌。肠壁一站（10个）、中间组（8个）淋巴结未查见癌。，免疫组化染色示：ERCC1弥漫（+）、TS少部分弱（+）、SYN（-）、CGA（-）。术后查无化疗禁忌后给予3周期化疗，，方案为：奥沙利铂150MG D1，亚叶酸钙0.3G+替加氟1.0G D2-D6，同时给与升白细胞、护肝、止吐、免疫增强治疗，患者副反应轻。院外期间患者一般情况好，无恶心，无腹痛腹胀胀不适，无现患者为行复查及化疗再次来院就诊，门诊以“直肠癌术后”收入院。   近期患者精神可，饮食可，大便正常，小便正常，近期体重无明显变化。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
请用中文回答
### Input:
现在有一个中文电子病历，请你帮我完成实体自动标注的任务，请注意，所需标注的实体类型为｛"疾病和诊断": 1, "手术": 2, "解剖部位": 3, "药物": 4, "影像检查": 5, "实验室检查": 6｝，电子病历的原文本为，患者3月前因“直肠癌”于在我院于全麻上行直肠癌根治术（DIXON术），手术过程顺利，术后给予抗感染及营养支持治疗，患者恢复好，切口愈合良好。，术后病理示：直肠腺癌（中低度分化），浸润溃疡型，面积3.5*2CM，侵达外膜。双端切线另送“近端”、“远端”及环周底部切除面未查见癌。肠壁一站（10个）、中间组（8个）淋巴结未查见癌。，免疫组化染色示：ERCC1弥漫（+）、TS少部分弱（+）、SYN（-）、CGA（-）。术后查无化疗禁忌后给予3周期化疗，，方案为：奥沙利铂150MG D1，亚叶酸钙0.3G+替加氟1.0G D2-D6，同时给与升白细胞、护肝、止吐、免疫增强治疗，患者副反应轻。院外期间患者一般情况好，无恶心，无腹痛腹胀胀不适，无现患者为行复查及化疗再次来院就诊，门诊以“直肠癌术后”收入院。   近期患者精神可，饮食可，大便正常，小便正常，近期体重无明显变化。
### Response:
#### Q1: 1. 2. 3. 4. 5. 6.
#### Q2: 1. 2. 3. 4. 5. 6.
#### Q3: 1. 2. 3. 4. 5. 6.
#### Q4: 1. 2. 3. 4. 5. 6.
#### Q5: 1. 2. 3. 4. 5. 6.
#### Q6: 1. 2. 3. 4. 5


In [ ]:
#4超参数设置&数据集加载，这里用的是先训练huatuo后训练Training
EOS_TOKEN = tokenizer.eos_token  # 必须添加 EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 必须添加EOS_TOKEN，否则无限生成
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import torch

# 加载第一个数据集
dataset_huatuo = load_dataset("veekypilia/huatuo_chinese", split="train")
dataset_huatuo = dataset_huatuo.map(formatting_prompts_func, batched=True)

# 加载第二个数据集
dataset_true_training = load_dataset("veekypilia/New_with_text", split="train")
dataset_true_training = dataset_true_training.map(formatting_prompts_func, batched=True)

# 定义一个函数，用于在训练时分别处理两个数据集
def train_model(dataset1, dataset2, model, tokenizer, max_seq_length, output_dir):
    # 设置训练参数
    training_args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=200,  # 微调步数
        learning_rate=2e-4,  # 学习率
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=output_dir,
    )

    # 训练第一个数据集
    trainer1 = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset1,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,  # 可以让短序列的训练速度提高5倍。
        args=training_args,
    )
    trainer1.train()

    # 训练第二个数据集
    trainer2 = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset2,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,  # 可以让短序列的训练速度提高5倍。
        args=training_args,
    )
    trainer2.train()

# 模型设置
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 建议 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.7,
    bias="none",
    use_gradient_checkpointing="unsloth",  # 检查点，长上下文度
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# 开始训练
train_model(dataset_huatuo, dataset_true_training, model, tokenizer, max_seq_length, "outputs")

pass

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.7.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

Step,Training Loss
1,2.634200
2,2.483600
3,2.688100
4,2.396100
5,2.416400
6,2.338500
7,2.272600
8,1.980500
9,2.147600
10,1.935600


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.524600
2,1.460300
3,1.412900
4,1.373900
5,1.491800
6,1.228900
7,1.193000
8,1.140500
9,1.121800
10,1.090200


In [ ]:
#版本查询
# Check CPU
!cat /proc/cpuinfo | grep 'model name\|processor'

# Check GPU
!nvidia-smi

# Check CUDA
!nvcc --version

processor	: 0
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
processor	: 1
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
Fri Jun 28 02:30:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0              29W /  70W |   7915MiB / 15360MiB |      0%      Default |
|                                         

In [ ]:
#5.1测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查，请同时标出你所标注的实体的类型，并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者2008年9月3日因“腹胀，发现腹部包块”在我院腹科行手术探查，术中见盆腹腔肿物，与肠管及子宫关系密切，遂行“全子宫左附件切除+盆腔肿物切除+右半结肠切除+DIXON术”，术后病理示颗粒细胞瘤，诊断为颗粒细胞瘤IIIC期，术后自2008年11月起行BEP方案化疗共4程，末次化疗时间为2009年3月26日。之后患者定期复查，2015-6-1，复查CT示：髂嵴水平上腹部L5腰椎前见软组织肿块，大小约30MM×45MM，密度欠均匀，边界尚清楚，轻度强化。查肿瘤标志物均正常。于2015-7-6行剖腹探查+膀胱旁肿物切除+骶前肿物切除+肠表面肿物切除术，术程顺利，，术后病理示：膀胱旁肿物及骶前肿物符合颗粒细胞瘤。于2015-7-13、8-14给予泰素240MG+伯尔定600MG化疗2程，过程顺利。出院至今，无发热，无腹痛、腹胀，有脱发，现返院复诊，拟行再次化疗收入院。起病以来，精神、胃纳、睡眠可，大小便正常，体重无明显改变。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查，请同时标出你所标注的实体的类型，并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者2008年9月3日因“腹胀，发现腹部包块”在我院腹科行手术探查，术中见盆腹腔肿物，与肠管及子宫关系密切，遂行“全子宫左附件切除+盆腔肿物切除+右半结肠切除+DIXON术”，术后病理示颗粒细胞瘤，诊断为颗粒细胞瘤IIIC期，术后自2008年11月起行BEP方案化疗共4程，末次化疗时间为2009年3月26日。之后患者定期复查，2015-6-1，复查CT示：髂嵴水平上腹部L5腰椎前见软组织肿块，大小约30MM×45MM，密度欠均匀，边界尚清楚，轻度强化。查肿瘤标志物均正常。于2015-7-6行剖腹探查+膀胱旁肿物切除+骶前肿物切除+肠表面肿物切除术，术程顺利，，术后病理示：膀胱旁肿物及骶前肿物符合颗粒细胞瘤。于2015-7-13、8-14给予泰素240MG+伯尔定600MG化疗2程，过程顺利。出院至今，无发热，无腹痛、腹胀，有脱发，现返院复诊，拟行再次化疗收入院。起病以来，精神、胃纳、睡眠可，大小便正常，体重无明显改变。
### Response:
[{'end_pos': 14, 'label_type': '解剖部位', 'overlap': 0, start_pos': 13, 'text': '腹'}, {'end_pos': 19, 'label_type': '解剖部位', 'overlap': 0, start_pos': 17, 'text': '腹部'}, {'end_pos': 52, 'label_type': '手术', 'overlap': 0, start_pos': 37, 'text': '全子宫左

In [ ]:
#5.2测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者于2011年9月29日在我院因“子宫内膜癌II期”在全麻上行“广泛全子宫切除+两侧附件切除+盆腔淋巴结清扫+腹主动脉旁淋巴结活检术”，术中探查见盆腹腔未见腹水，子宫增大，约10*8*7CM，饱满，两侧附件未见异常，盆腔及腹主动脉旁淋巴结未及肿大。术程顺利，，术后病理回报：腹水未见癌；（全子宫+两附件）送检子宫大小为10*6*4CM，宫腔内见菜花样肿物大小为5*4*3CM，灰黄质硬，浸润浅肌层；镜上中至低分化子宫内膜样腺癌，部分呈鳞状分化，浸润子宫浅肌层，未累及宫颈管；右输卵管系膜内见子宫内膜异位；两附件、阴道残端、淋巴结未见癌；，免疫组化：ER（+），PR（-）。，术后诊断：子宫内膜样腺癌IA1期。因肿瘤为中至低分化且大小为5*4*3CM，术后有化疗指征。于2011年10月11日、11月16日行TP（泰素+伯尔定）方案化疗2程，化疗后出现轻度恶心、呕吐，伴脱发，无骨痛及四肢麻木等不适，白细胞最低降至2.7×109/L，未处理可自行升至正常。自发病以来，精神、食欲、睡眠良好，无腹痛及腹胀，无腰酸，大小便正常。体重较下次化疗增加3KG。，既往化疗及肿瘤标志物情况：，化疗药物毒副反应：。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者于2011年9月29日在我院因“子宫内膜癌II期”在全麻上行“广泛全子宫切除+两侧附件切除+盆腔淋巴结清扫+腹主动脉旁淋巴结活检术”，术中探查见盆腹腔未见腹水，子宫增大，约10*8*7CM，饱满，两侧附件未见异常，盆腔及腹主动脉旁淋巴结未及肿大。术程顺利，，术后病理回报：腹水未见癌；（全子宫+两附件）送检子宫大小为10*6*4CM，宫腔内见菜花样肿物大小为5*4*3CM，灰黄质硬，浸润浅肌层；镜上中至低分化子宫内膜样腺癌，部分呈鳞状分化，浸润子宫浅肌层，未累及宫颈管；右输卵管系膜内见子宫内膜异位；两附件、阴道残端、淋巴结未见癌；，免疫组化：ER（+），PR（-）。，术后诊断：子宫内膜样腺癌IA1期。因肿瘤为中至低分化且大小为5*4*3CM，术后有化疗指征。于2011年10月11日、11月16日行TP（泰素+伯尔定）方案化疗2程，化疗后出现轻度恶心、呕吐，伴脱发，无骨痛及四肢麻木等不适，白细胞最低降至2.7×109/L，未处理可自行升至正常。自发病以来，精神、食欲、睡眠良好，无腹痛及腹胀，无腰酸，大小便正常。体重较下次化疗增加3KG。，既往化疗及肿瘤标志物情况：，化疗药物毒副反应：。
### Response:
[{'end_pos': 16, 'label_type': '疾病和诊断', 'overlap': 0, start_pos': 12, 'text': '子宫内膜癌II期'}, {'end_pos': 38, 'label_type': '手术', 'overlap': 0, start_pos': 24, 'text': '广泛全子宫切除+两侧附

In [ ]:
#5.3测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者于2010年10月因\"上腹痛伴大便习惯改变\"外院行肠镜，活检病理示中分化腺癌；，外院B超示：盆腔内2个巨大团块，考虑卵巢来源可能性大；盆腔大量积液。来我院就诊，考虑乙状结肠癌伴不完全梗阻、盆腔肿物，2010-10-25我院行\"子宫切除+两附件切除+DIXON术+大网膜切除术开腹恶性肿瘤特殊治疗术\"，术中5-FU1000MG肠腔化疗，门静脉5-FU250MG化疗，中人氟安800MG腹腔化疗，，病理回报：乙状结肠中分化腺癌，两侧卵巢转移瘤，中央组淋巴结转移4/15，PT4BN2M1。2010-11-10我院复查CT提示肝S5、6段肝转移。遂于2010-11-16至2011-3-11行FORFIRI方案化疗7程，4程、6程复查CT疗效评价为SD，后于2011-4-6转外科行\"肝转移瘤切除术\"，，术后病理：符合肠癌肝转移。后于2011-05-20至2011-08-05继续FORFIRI方案化疗4程。后定期复查至2012-02-21我院CT提示右下肺及右上肺结节、左前胸壁结节，考虑转移瘤，两肺多发结节结节状小空洞影未排除转移。考虑肿瘤复发，于2012-03-01始行FOLFOX方案6程，3，程后复查CT疗效评价：SD。，6程后2012-7-10我院CT：肝S6、8、2病灶，可疑转移瘤，建议MR检查。脾内低密度影，可疑转移瘤。，疗效评价：SD。现为进一步诊治入院，患者自觉无不适，胃纳、睡眠可，二便如常。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者于2010年10月因"上腹痛伴大便习惯改变"外院行肠镜，活检病理示中分化腺癌；，外院B超示：盆腔内2个巨大团块，考虑卵巢来源可能性大；盆腔大量积液。来我院就诊，考虑乙状结肠癌伴不完全梗阻、盆腔肿物，2010-10-25我院行"子宫切除+两附件切除+DIXON术+大网膜切除术开腹恶性肿瘤特殊治疗术"，术中5-FU1000MG肠腔化疗，门静脉5-FU250MG化疗，中人氟安800MG腹腔化疗，，病理回报：乙状结肠中分化腺癌，两侧卵巢转移瘤，中央组淋巴结转移4/15，PT4BN2M1。2010-11-10我院复查CT提示肝S5、6段肝转移。遂于2010-11-16至2011-3-11行FORFIRI方案化疗7程，4程、6程复查CT疗效评价为SD，后于2011-4-6转外科行"肝转移瘤切除术"，，术后病理：符合肠癌肝转移。后于2011-05-20至2011-08-05继续FORFIRI方案化疗4程。后定期复查至2012-02-21我院CT提示右下肺及右上肺结节、左前胸壁结节，考虑转移瘤，两肺多发结节结节状小空洞影未排除转移。考虑肿瘤复发，于2012-03-01始行FOLFOX方案6程，3，程后复查CT疗效评价：SD。，6程后2012-7-10我院CT：肝S6、8、2病灶，可疑转移瘤，建议MR检查。脾内低密度影，可疑转移瘤。，疗效评价：SD。现为进一步诊治入院，患者自觉无不适，胃纳、睡眠可，二便如常。
### Response:
[{'end_pos': 12, 'label_type': '解剖部位', 'overlap': 0, start_pos': 10, '

In [ ]:
#5.4测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者因\"体检发现子宫肌瘤\"于2008年5月就诊我院， 2008年5月4日于我院行全宫两附件切除附件切除术，，术后病理为:子宫平滑肌肉瘤,术后于2008.5.9～2008.8.7行EPI+IFO+DTIC方案化疗四程(均为静脉用药），化疗过程顺利。出院后定期复查，2011-6-2胸片检查提示左肺结节影，考虑转移瘤可能性大，遂进一步CT检查，，提示：子宫肉瘤术后改变，未见明确复发征。两肺多发结节，考虑肺转移瘤。于2011-6-27、2011-7-22、2011-8-19、2011-9-23、2011-10-25及2011-11-22予“IA+DTIC”化疗六程，第一次化疗过程顺利，第二次化疗的最后一天，患者诉胸闷，予暂停化疗药物。化疗后出现轻度胃肠道反应。第六次化疗后白细胞最低降至2.5×109/L，升白细胞治疗后好转，，患者于第六次化疗前复查CT提示：两肺转移瘤较前无明显改变；2012-2-28入院予“里葆多40MG D1+匹服平3.0 D1-2+安维汀300MGD1”化疗一程，WBC最低至2.3×109/L，升白治疗后好转。现患者再次入院化疗，精神、睡眠好，食欲欠佳，大小便正常。近期体重无明显改变。，既往化疗及肿瘤标志物情况：，化疗药物毒副反应：。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者因"体检发现子宫肌瘤"于2008年5月就诊我院， 2008年5月4日于我院行全宫两附件切除附件切除术，，术后病理为:子宫平滑肌肉瘤,术后于2008.5.9～2008.8.7行EPI+IFO+DTIC方案化疗四程(均为静脉用药），化疗过程顺利。出院后定期复查，2011-6-2胸片检查提示左肺结节影，考虑转移瘤可能性大，遂进一步CT检查，，提示：子宫肉瘤术后改变，未见明确复发征。两肺多发结节，考虑肺转移瘤。于2011-6-27、2011-7-22、2011-8-19、2011-9-23、2011-10-25及2011-11-22予“IA+DTIC”化疗六程，第一次化疗过程顺利，第二次化疗的最后一天，患者诉胸闷，予暂停化疗药物。化疗后出现轻度胃肠道反应。第六次化疗后白细胞最低降至2.5×109/L，升白细胞治疗后好转，，患者于第六次化疗前复查CT提示：两肺转移瘤较前无明显改变；2012-2-28入院予“里葆多40MG D1+匹服平3.0 D1-2+安维汀300MGD1”化疗一程，WBC最低至2.3×109/L，升白治疗后好转。现患者再次入院化疗，精神、睡眠好，食欲欠佳，大小便正常。近期体重无明显改变。，既往化疗及肿瘤标志物情况：，化疗药物毒副反应：。
### Response:
[{'end_pos': 17, 'label_type': '疾病和诊断', 'overlap': 0, start_pos': 13, 'text': '子宫肌瘤'}, {'end_pos': 59, 'label_type': '手术', 'overlap': 0, start_po

In [ ]:
#5.5测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，2009年4月行乙状结肠癌根治术，，术后病理：中分化腺癌（PT3N0M0），术后未行放化疗。2010年4月复查发现左下肺及肝转移，行草酸铂+希罗达化疗2周期，2010年7月行肝转移灶切除术。术后原方案继续化疗3周期。继续口服3周期希罗达单药。每3月复查。2011-6-28复查肿标正常，胸腹CT所见均与前相仿。B，超：右颈根部及右锁骨下多发淋巴结，大者1.2X0.5CM。近期进食可、术后至今体重增加7KG。2011.10复查较前(2011.7)无明显变化,肿瘤标志物(-) 。此次，取2012.1.12复查腹部CT、肺CT结果，提示肺部小病灶同前，肝脏未见异常包块。肿标（-）。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，2009年4月行乙状结肠癌根治术，，术后病理：中分化腺癌（PT3N0M0），术后未行放化疗。2010年4月复查发现左下肺及肝转移，行草酸铂+希罗达化疗2周期，2010年7月行肝转移灶切除术。术后原方案继续化疗3周期。继续口服3周期希罗达单药。每3月复查。2011-6-28复查肿标正常，胸腹CT所见均与前相仿。B，超：右颈根部及右锁骨下多发淋巴结，大者1.2X0.5CM。近期进食可、术后至今体重增加7KG。2011.10复查较前(2011.7)无明显变化,肿瘤标志物(-) 。此次，取2012.1.12复查腹部CT、肺CT结果，提示肺部小病灶同前，肝脏未见异常包块。肿标（-）。
### Response:
[{'end_pos': 16, 'label': '乙状结肠癌', 'overlap': 0, start_pos': 11}, {'end_pos': 33, 'label': '中分化腺癌', 'overlap': 0, start_pos': 28}, {'end_pos': 58, 'label': '左下肺', 'overlap': 0, start_pos': 57}, {'end_pos': 61, 'label': '肝', 'overlap': 0, start_pos': 60}, {'end_pos': 71, 'label': '草酸铂', 'overlap': 0, start_pos': 68}, {'end_pos': 75, 'label': '希罗达', 'overlap': 0, start_pos': 72}, {'end_

In [ ]:
#5.6测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者因无明显诱因出现血便，鲜红色，量少，大便次数稍增多，无里急后重及排便不尽感，无恶心呕吐，无腹胀腹痛等不适。，在外院行肠镜检查示：距肛30CM环肠周肿物，肠镜勉强通过，，病理结果为：乙状结肠高分化腺癌。为进一步诊治来我院门诊就诊，门诊以“乙状结肠腺癌”收入结直肠科，2013-01-30，行CT提示：乙状结肠肿物，考虑乙状结肠癌可能性大；右侧髂总动脉淋巴结肿大，考虑为淋巴结转移；乙状结肠系膜及腹主动脉旁多发小淋巴结，考虑淋巴结转移可能性大；肝S5 稍低密度结节，考虑转移瘤可能性大；肝S8 低密度灶，考虑为肝囊肿。于2013-02-04在全麻上行乙状结肠癌切除术。术中探查见肿瘤位于降结肠上段，侵及浆膜、肠系膜血管根部血管根部淋巴结肿大侵润，大小约1.5厘米，小肠系膜见淋巴结，冰冻2枚，未见转移。术后分期PT3N2AM1X,IV期。术后于2013-03-04给予XELOX方案化疗1程顺利,胃肠道反应I°，神经毒性I°。现返院行上一程治疗。起病以来，无发热、咳嗽、身目黄染，无骶部痛等，精神可，胃纳、睡眠可，小便无异常，近期体重上降约5KG。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者因无明显诱因出现血便，鲜红色，量少，大便次数稍增多，无里急后重及排便不尽感，无恶心呕吐，无腹胀腹痛等不适。，在外院行肠镜检查示：距肛30CM环肠周肿物，肠镜勉强通过，，病理结果为：乙状结肠高分化腺癌。为进一步诊治来我院门诊就诊，门诊以“乙状结肠腺癌”收入结直肠科，2013-01-30，行CT提示：乙状结肠肿物，考虑乙状结肠癌可能性大；右侧髂总动脉淋巴结肿大，考虑为淋巴结转移；乙状结肠系膜及腹主动脉旁多发小淋巴结，考虑淋巴结转移可能性大；肝S5 稍低密度结节，考虑转移瘤可能性大；肝S8 低密度灶，考虑为肝囊肿。于2013-02-04在全麻上行乙状结肠癌切除术。术中探查见肿瘤位于降结肠上段，侵及浆膜、肠系膜血管根部血管根部淋巴结肿大侵润，大小约1.5厘米，小肠系膜见淋巴结，冰冻2枚，未见转移。术后分期PT3N2AM1X,IV期。术后于2013-03-04给予XELOX方案化疗1程顺利,胃肠道反应I°，神经毒性I°。现返院行上一程治疗。起病以来，无发热、咳嗽、身目黄染，无骶部痛等，精神可，胃纳、睡眠可，小便无异常，近期体重上降约5KG。
### Response:
[{'end_pos': 29, 'label_type': '解剖部位', 'overlap': 0, start_pos': 28}, {'end_pos': 44, 'label_type': '解剖部位', 'overlap': 0, start_pos': 43}, {'end_pos': 51, 'label_type': '解剖部位', 'overlap': 0, start_pos': 

In [ ]:
#5.7测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者于4月余前无明显诱因出现腹胀不适，与饮食无明显相关，伴泛酸、嗳气，伴恶心、呕吐，为胃内容物，无呕血，无腹泻、腹胀，无黑便、便血、粘液便，，于外院行胃镜检查示：胃窦5×5㎝肿物。，病理：低分化腺癌。，CT：符合胃窦癌。胃小弯侧、幽门上方淋巴结肿大。左肺中叶少许炎症。脾稍大。肝、胆、胰、两肾和盆腔未见病变。2015-08-25手术室全麻上行胃癌根治术，2015-09-11、2015-10-22、2015-11-13及2015-12-07行SOX方案化疗4程，现按主诊教授意见入院行上次治疗。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者于4月余前无明显诱因出现腹胀不适，与饮食无明显相关，伴泛酸、嗳气，伴恶心、呕吐，为胃内容物，无呕血，无腹泻、腹胀，无黑便、便血、粘液便，，于外院行胃镜检查示：胃窦5×5㎝肿物。，病理：低分化腺癌。，CT：符合胃窦癌。胃小弯侧、幽门上方淋巴结肿大。左肺中叶少许炎症。脾稍大。肝、胆、胰、两肾和盆腔未见病变。2015-08-25手术室全麻上行胃癌根治术，2015-09-11、2015-10-22、2015-11-13及2015-12-07行SOX方案化疗4程，现按主诊教授意见入院行上次治疗。
### Response:
[{'end_pos': 15, 'label': '解剖部位', 'overlap': 0, start_pos': 14}, {'end_pos': 37, 'label': '解剖部位', 'overlap': 0, start_pos': 36}, {'end_pos': 56, 'label': '解剖部位', 'overlap': 0, start_pos': 55}, {'end_pos': 64, 'label': '解剖部位', 'overlap': 0, start_pos': 63}, {'end_pos': 72, 'label': '解剖部位', 'overlap': 0, start_pos': 71}, {'end_pos': 103, 'label': '解剖部位', 'overlap': 0, start_pos': 100}, {'end_pos': 123, 'label': '解剖部位', 'overlap'

In [ ]:
#5.8测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者9月余前因反复黑便伴腹部不适于我院消化内科住院，，查血常规示：WBC 3.55*10^9/L，RBC 3.05*10^12/L，HB 66G/L，PLT 290*10^9/L；，电子胃镜示：贲门炎症；，电子小肠镜示：空回肠交界处黏膜病变性质待查；，小肠病变病理示：（空回肠交界）粘膜内可见小片异型细胞，符合低分化腺癌；，免疫组化示：CK7(-),CK20(+),CK5/6(-)。于2015-06-13转入普外科，排除手术禁忌后行全麻上小肠部分切除术，，术后病理示：（小肠）T细胞淋巴瘤，结合免疫组化，符合肠病相关性T细胞淋巴瘤（I型），术后转入我科，于2015-8-13、2015-9-4给予CHOP-E方案化疗2周期，患者未再黑便，腹部不适明显好转。于2015-10-11、2015-11-9给予2周期CHOP-E方案化疗，，复查CT示：两侧颈部小淋巴结；肝脏钙化灶；胆囊底部壁局限性增厚，较2015-10-09CT片变化不明显；右侧肾下腺饱满；右肾局限性略低强化，建议随访复查；左侧肾盂及左侧输尿管中下段扩张，右侧输尿管局部可见，请结合超声；腹腔及腹膜后多发小淋巴结；结合病史，小肠术后，局部肠壁厚，请结合临床及其它检查；子宫强化欠均匀，两侧附件区饱满，请结合超声；盆腔钙化结节，较前片变化不明显。效果欠佳。后结合患者及其家属意愿于2015-12-4、2016-01-01、2016-1-28给予GDP方案化疗3周期,耐受可。现为行进一步治疗再次收入我科。患者自下次出院以来，饮食睡眠可，大小便正常，体重无明显变化。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者9月余前因反复黑便伴腹部不适于我院消化内科住院，，查血常规示：WBC 3.55*10^9/L，RBC 3.05*10^12/L，HB 66G/L，PLT 290*10^9/L；，电子胃镜示：贲门炎症；，电子小肠镜示：空回肠交界处黏膜病变性质待查；，小肠病变病理示：（空回肠交界）粘膜内可见小片异型细胞，符合低分化腺癌；，免疫组化示：CK7(-),CK20(+),CK5/6(-)。于2015-06-13转入普外科，排除手术禁忌后行全麻上小肠部分切除术，，术后病理示：（小肠）T细胞淋巴瘤，结合免疫组化，符合肠病相关性T细胞淋巴瘤（I型），术后转入我科，于2015-8-13、2015-9-4给予CHOP-E方案化疗2周期，患者未再黑便，腹部不适明显好转。于2015-10-11、2015-11-9给予2周期CHOP-E方案化疗，，复查CT示：两侧颈部小淋巴结；肝脏钙化灶；胆囊底部壁局限性增厚，较2015-10-09CT片变化不明显；右侧肾下腺饱满；右肾局限性略低强化，建议随访复查；左侧肾盂及左侧输尿管中下段扩张，右侧输尿管局部可见，请结合超声；腹腔及腹膜后多发小淋巴结；结合病史，小肠术后，局部肠壁厚，请结合临床及其它检查；子宫强化欠均匀，两侧附件区饱满，请结合超声；盆腔钙化结节，较前片变化不明显。效果欠佳。后结合患者及其家属意愿于2015-12-4、2016-01-01、2016-1-28给予GDP方案化疗3周期,耐受可。现为行进一步治疗再次收入我科。患者自下次出院以来，饮食睡眠可，大小便正常，体重无明显变化。
### Response:
[{'end_pos': 2

In [ ]:
#5.9测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者2012年2月2日因“子宫内膜癌”行筋膜外子宫全切、两侧卵巢输卵管切除及盆腔淋巴结清扫术。，手术评价：手术范围充分，无残留肉眼病灶。，腹水查瘤细胞：少量异型增生细胞考虑肿瘤细胞。，术后病理：子宫内膜样腺癌I级，侵及肌层大于1/2,余未见异常。，术后诊断：子宫内膜癌IB期（高分化），术后行表阿霉素+顺铂化疗4个疗程，末次化疗时间2012-5-30，过程顺利。现为求补充治疗收入我科。患者出院至今，无头晕心慌，无发热，无腹胀腹痛，饮食睡眠良，患者于第3次化疗时复查盆腔彩超，提示盆腔囊性回声4厘米，而后规律复查，囊肿持续存在，CA125持续正常。3个月前门诊复查超声时提示囊肿转为低回声，边界欠清晰，边缘见彩色血流，口服西黄丸及红金消结胶囊后复查超声无明显变化。现患者为求进一步诊治入院。术后化疗至今一般状态良好，偶有左上腹隐痛，程度轻，可自行缓解，大小便正常，体重未见明显改变。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者2012年2月2日因“子宫内膜癌”行筋膜外子宫全切、两侧卵巢输卵管切除及盆腔淋巴结清扫术。，手术评价：手术范围充分，无残留肉眼病灶。，腹水查瘤细胞：少量异型增生细胞考虑肿瘤细胞。，术后病理：子宫内膜样腺癌I级，侵及肌层大于1/2,余未见异常。，术后诊断：子宫内膜癌IB期（高分化），术后行表阿霉素+顺铂化疗4个疗程，末次化疗时间2012-5-30，过程顺利。现为求补充治疗收入我科。患者出院至今，无头晕心慌，无发热，无腹胀腹痛，饮食睡眠良，患者于第3次化疗时复查盆腔彩超，提示盆腔囊性回声4厘米，而后规律复查，囊肿持续存在，CA125持续正常。3个月前门诊复查超声时提示囊肿转为低回声，边界欠清晰，边缘见彩色血流，口服西黄丸及红金消结胶囊后复查超声无明显变化。现患者为求进一步诊治入院。术后化疗至今一般状态良好，偶有左上腹隐痛，程度轻，可自行缓解，大小便正常，体重未见明显改变。
### Response:
[{'end_pos': 15, 'label_type': '疾病和诊断', 'overlap': 0, start_pos': 11, 'text': '子宫内膜癌'}, {'end_pos': 37, 'label_type': '手术', 'overlap': 0, start_pos': 17, 'text': '筋膜外子宫全切、两侧卵巢输卵管切除及盆腔淋巴结清扫术'}, {'end_pos': 57, 'label_type': '解剖部位', 'overlap': 0, start_pos': 55, 'text': '腹水'}, {'end_pos'

In [ ]:
#5.10（其实是11）测试微调后的模型
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者8月余前始无明显诱因出现进食后下腹胀痛，以进食硬物时明显，持续10余分钟后可自行缓解，伴返流，偶伴嗳气，无放射性痛，无心悸、胸闷、心前区疼痛，无呕吐、返酸、呕血，无吞咽困难、呛咳，无厌油、身目黄染，无解黑便、血便，未予特殊治疗。后症状加重，为进一步诊治到我院就诊， 2012-03-02，我院电子胃镜示：胃癌（皮革胃）侵犯贲门；，病理活检示：低分化腺癌。2012-03-05，我院下腹部CT示：贲门、胃底胃壁明显增厚，符合贲门胃底癌。肝S6低密度灶，考虑小囊肿可能性大。于2012-03-08、2012-04-05、2012-04-27三次在我院内科予“草酸铂+替吉奥”联合化疗。于2012－5－30我院行“根治性全胃切除术”，术后病理（493573）示低分化腺癌浸润至浆膜上及食管浅肌层，近端（＋），淋巴结（＋）4/16。术后恢复可，于2012－07－19,2012-9-20、2012-10-18、2012-11-15给予\" 替吉奥+多西他赛\"方案化疗4程,2012－12－13改为替吉奥单药维持.今日遵教授医嘱，继续替吉奥单药维持，现为进一步治疗入院。起病以来，患者无畏寒发热，无咳嗽、咳痰，胃纳精神尚可，二便正常，体重近4个月上降15KG。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
只用中文回答问题
### Input:
请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者8月余前始无明显诱因出现进食后下腹胀痛，以进食硬物时明显，持续10余分钟后可自行缓解，伴返流，偶伴嗳气，无放射性痛，无心悸、胸闷、心前区疼痛，无呕吐、返酸、呕血，无吞咽困难、呛咳，无厌油、身目黄染，无解黑便、血便，未予特殊治疗。后症状加重，为进一步诊治到我院就诊， 2012-03-02，我院电子胃镜示：胃癌（皮革胃）侵犯贲门；，病理活检示：低分化腺癌。2012-03-05，我院下腹部CT示：贲门、胃底胃壁明显增厚，符合贲门胃底癌。肝S6低密度灶，考虑小囊肿可能性大。于2012-03-08、2012-04-05、2012-04-27三次在我院内科予“草酸铂+替吉奥”联合化疗。于2012－5－30我院行“根治性全胃切除术”，术后病理（493573）示低分化腺癌浸润至浆膜上及食管浅肌层，近端（＋），淋巴结（＋）4/16。术后恢复可，于2012－07－19,2012-9-20、2012-10-18、2012-11-15给予" 替吉奥+多西他赛"方案化疗4程,2012－12－13改为替吉奥单药维持.今日遵教授医嘱，继续替吉奥单药维持，现为进一步治疗入院。起病以来，患者无畏寒发热，无咳嗽、咳痰，胃纳精神尚可，二便正常，体重近4个月上降15KG。
### Response:
[{'end_pos': 21, 'label_type': '解剖部位', 'overlap': 0, start_pos': 18, 'text': '下腹'}, {'end_pos': 67, 'label_type': '解剖部位', 'overlap': 0, start_pos': 66,

In [ ]:
#6保存LoRA模型
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # 在线保存到hugging face，需要token

In [ ]:
#8挂载谷歌云盘
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [ ]:
#9复制模型到google drive
import shutil
import os

source_file = '/content/model/tokenizer_config.json'
if os.path.exists(source_file):
    destination_dir = '/content/drive/MyDrive/Llama3_whole_loraed'

    os.makedirs(destination_dir, exist_ok=True)
    destination_file = f'{destination_dir}/tokenizer_config.json'
    shutil.copy(source_file, destination_file)
    print("文件已成功保存到谷歌云盘")
else:
    print(f"目标文件找不到: {source_file}")

文件已成功保存到谷歌云盘


In [ ]:
#尝试挂载谷歌云盘中的模型，但是有点问题
!pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer # Use AutoModelForCausalLM for decoder-only models
model_path = '/content/drive/MyDrive/Llama3_trained'  # 模型在Google Drive中的路径
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答问题", # instruction
        "请你来完成实体识别的任务，请注意，实体类型共有：疾病和诊断、手术、药物、解剖部位、实验室检查、影像检查，请同时标出你所标注的实体的类型，并依据你标注的每个实体的位置索引输出对应的实体原文本。需要你识别的电子病历的原文本为：，患者2008年9月3日因“腹胀，发现腹部包块”在我院腹科行手术探查，术中见盆腹腔肿物，与肠管及子宫关系密切，遂行“全子宫左附件切除+盆腔肿物切除+右半结肠切除+DIXON术”，术后病理示颗粒细胞瘤，诊断为颗粒细胞瘤IIIC期，术后自2008年11月起行BEP方案化疗共4程，末次化疗时间为2009年3月26日。之后患者定期复查，2015-6-1，复查CT示：髂嵴水平上腹部L5腰椎前见软组织肿块，大小约30MM×45MM，密度欠均匀，边界尚清楚，轻度强化。查肿瘤标志物均正常。于2015-7-6行剖腹探查+膀胱旁肿物切除+骶前肿物切除+肠表面肿物切除术，术程顺利，，术后病理示：膀胱旁肿物及骶前肿物符合颗粒细胞瘤。于2015-7-13、8-14给予泰素240MG+伯尔定600MG化疗2程，过程顺利。出院至今，无发热，无腹痛、腹胀，有脱发，现返院复诊，拟行再次化疗收入院。起病以来，精神、胃纳、睡眠可，大小便正常，体重无明显改变。", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/drive/MyDrive/Llama3_trained.